<a href="https://colab.research.google.com/github/hinekobot/Tensorflow2.0_sample/blob/master/01_LinearRegressionSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow2.0  
- Google Colab上での操作を想定
- cloneして中身を変える用の空ファイル

In [7]:
!pip install tensorflow==2.0.0-alpha0

    100% |████████████████████████████████| 79.9MB 458kB/s 
    100% |████████████████████████████████| 3.0MB 8.6MB/s 
    100% |████████████████████████████████| 61kB 25.1MB/s 
    100% |████████████████████████████████| 419kB 18.5MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [1]:
import tensorflow as tf
tf.version.VERSION

'2.0.0-alpha0'

In [0]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y +2


## Linear Regression
### by Using Normal Equation

In [8]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
#theta = tf.matmul(tf.matmul(tf.matrix_inverse(ft.matmul(XT, X)), XT), y)
theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)

I0401 00:55:29.908049 139903384319872 california_housing.py:114] Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /root/scikit_learn_data


In [9]:
theta

<tf.Tensor: id=2724, shape=(9, 1), dtype=float32, numpy=
array([[-3.7144665e+01],
       [ 4.3637392e-01],
       [ 9.3933418e-03],
       [-1.0714764e-01],
       [ 6.4542186e-01],
       [-4.1016874e-06],
       [-3.7816020e-03],
       [-4.2341197e-01],
       [-4.3682083e-01]], dtype=float32)>

### Linear Regression by Using Stochastic Descent
- 解説書だと`tf.assign()`を使っているが、Tensorflow2.0では削除されている
- 仕方ないのでループで計算させている

In [0]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
#scaler = StandardScaler()
scaler = MinMaxScaler()

scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [11]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
#theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  theta = theta - learning_rate * gradients
  y_pred = tf.matmul(X, theta,name='theta')
  error = y_pred - y
  mse = tf.reduce_mean(tf.square(error),name='mse')
  gradients = 2/m * tf.matmul(tf.transpose(X), error)

best_theta = theta.numpy()

Epoch 0 MSE= 4.6538177
Epoch 100 MSE= 1.6631339
Epoch 200 MSE= 1.4238535
Epoch 300 MSE= 1.3489833
Epoch 400 MSE= 1.289773
Epoch 500 MSE= 1.236963
Epoch 600 MSE= 1.1893682
Epoch 700 MSE= 1.1463536
Epoch 800 MSE= 1.1073896
Epoch 900 MSE= 1.0720185


### Linear Regression by Using Stochastic Descent: Auto differentiation
- 解説書だと`tf.gradients`を使用しているが`egar execution`では使えない
- `eager execution`では`tf.GradientTape()`を使用する
  - `tape.watch()`で変数を監視する
  - 監視しないとうまく動かない
- 微分する変数をリストで渡す
- 返り値(`gradients`)もリストになっている

In [105]:
n_epochs = 1000
learning_rate = 0.001

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  with tf.GradientTape() as g:
    g.watch(theta)
    y_pred = tf.matmul(X, theta,name='predictions')
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error),name='mse')

  gradients = g.gradient(mse, [theta])
  theta = theta - 0.01 * gradients[0]

best_theta = theta.numpy()

Epoch 0 MSE= 0.9298671
Epoch 100 MSE= 1.7388384
Epoch 200 MSE= 1.431847
Epoch 300 MSE= 1.363079
Epoch 400 MSE= 1.3122017
Epoch 500 MSE= 1.2661529
Epoch 600 MSE= 1.2237885
Epoch 700 MSE= 1.1847028
Epoch 800 MSE= 1.1485779
Epoch 900 MSE= 1.1151388


### Linear Regression by Using Stochastic Descent: Auto differentiation and optimizer


In [109]:
n_epochs = 1000
learning_rate = 0.001

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  with tf.GradientTape() as g:
    g.watch(theta)
    y_pred = tf.matmul(X, theta,name='predictions')
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error),name='mse')

  gradients = g.gradient(mse, [theta])
  optimizer.apply_gradients(zip(gradients, [theta]))
  #theta = theta - 0.01 * gradients[0]

best_theta = theta.numpy()

Epoch 0 MSE= 1.5605302
Epoch 100 MSE= 5.1014123
Epoch 200 MSE= 4.4591217
Epoch 300 MSE= 3.873743
Epoch 400 MSE= 3.345227
Epoch 500 MSE= 2.8735108
Epoch 600 MSE= 2.45852
Epoch 700 MSE= 2.1001532
Epoch 800 MSE= 1.7982754
Epoch 900 MSE= 1.5526927
